# Activities

In the previous lesson you learned how to map data structures into tables in a database through the EF (our chosen ORM). In this document you will learn how to access and interact with the data stored in the database.  



This document guides you through the main constructs of the Language Integrate Query (LINQ). LINQ is a a component of the .Net Framework that allows .Net languages to interact with objects (in our case the data stored in the database which are presented as objects by the EF) by means of a declarative syntax similar to Structured Query Language (SQL). As in SQL, LINQ operators in general can project, filter, etc. data from matrices , enumerable classes, XML, etc. As result of a query in LINQ, we have always a collection of objects that can be enumerated. Every query in LINQ is type-checked. This means that queries which envolve objects that are not compatible with each other will not be executed. When connected to a database LINQ queries are translated to their SQL respective at compile time.  


Although the EF reduces the complexity, performing queries in an application needs to be done carefully to avoid side-effects, such as opening and closing too often the connection with the DB, selecting more than necessary, queries that produce bad access plans, etc.


In the previous practicum we started to model a movie database, first with a 1<>Many relation and then with a Many<>Many relation. To avoid too much complexity and focus only on the topic of this lesson (LINQ), in the model of this practicum a `Movie` entity has a one-to-many relationship with an `Actor` entity. In the following we will see how to use LINQ to interact with our data. In the end we will see how to analyse the compiled queries to, for example, assess the performance. 


Note. For every LINQ query we will first mention its equivalent version in pseudo-SQL.

The implemented queries in this document are:
    - Projection 
    - Filtering          
    - Ordering       
    - Grouping                 
    - Joining
    - Subquery             
    - Aggregation (Count, Min, Max, Sum, Average)

### Model upgrade
- Extend the `Movie` model made so far to capture more data about our sample. This step is necessary to write more advanced queries. In the following you can see the new updated model



``` csharp
using System;
using System.Collections.Generic;
using Microsoft.EntityFrameworkCore;

namespace Model{
  public class MovieContext : DbContext {
        public DbSet<Movie> Movies { get; set; }
        public DbSet<Actor> Actors { get; set; }

        protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder){
          optionsBuilder.UseNpgsql("User ID=postgres;Password=;Host=localhost;Port=5432;Database=MovieDB;Pooling=true;");
        }
    }

    public class Movie {
        public int Id { get; set; }
        public string Title { get; set; }
        public DateTime Release { get; set; }
        public List<Actor> Actors { get; set; }
    }

    public class Actor {
        public int Id { get; set; }
        public string Name { get; set; }
        public DateTime Birth { get; set; }
        public string Gender { get; set; }
        public int MovieId { get; set; }
        public Movie Movie { get; set; }

    }
}
```

- You can add more attributes as long as you are using them correctly in your queries

- Since our model has changed we need to run a new migration and update to commit the changes to our db. In the command lines


``` bash
dotnet ef migrations add ImprovedModel
dotnet ef database update 
```

- Note. Remember to drop the database `dotnet ef database drop` in case the update call fails

### Setting up the query testing environment
- Modify the Program.cs file to reference to the LINQ library. This is done by adding `using System.Linq;` on top of the file. In this document all the implementation will be done in the Program.cs file

- We strongly advise to group queries in different static methods as shown in the following code to ease testing


``` csharp
...
class Program{

	static void Main(string[] args)
	{
		...
        DataInsertion();
        Projection();
        Join ();
        //etc.
		...
	}

    static void DataInsertion(){           
		using (var db = new MovieContext())
        {
            ...
		}
	}
    static void Projection(){           
		using (var db = new MovieContext())
        {
            ...
		}
	}
    static void Join(){           
        using (var db = new MovieContext())
        {
            ...
		}
	}
    //etc.
    
    ...
}
```


- Remember the usage of `using`. The `using` keyword takes care of closing the connection with the database once its body is executed

- Since the model changed, it is important to insert new data. Try also to add more than one movie and vary the number of actors for each to get different results. **Remember** to make your changes persistent in the database; this can only be done by calling the `SaveChanges()` method from the `DbContext`


``` csharp
...
static void DataInsertion()
{
    using (var db = new MovieContext()){
        Movie m = new Movie{
            Title = "Divorce Italian Style",
            Release = DateTime.Now,
            Actors = new System.Collections.Generic.List<Actor> {
                new Actor{ Name = "Marcello Mastroianni",
                            Birth = new DateTime(1988, 8, 29), 
                            Gender=  "Male",
                            },
                new Actor{ Name = "Daniela Rocca",
                            Birth = new DateTime(1986, 5, 1), 
                            Gender=  "Female",
                            }
                    }
            };

        //Add more movies here
        ...

        db.Add(m);
        db.SaveChanges();
        ...
    }
}
...
```

- In the command line execute the run command (`dotnet run`) to save this data. Remember, when you test the later functions (Projection, Join, ...), to comment the call to the DataInsertion function, otherwise the same data will be added and re-added at every run



### Projection
- In a projection we define a set of attributes which values need to be obtained as result. For example if we select the attributes `Title` and `Release` of all movies, as a result we will get only the values of this set of attributes from the database associated to all movies

  - SQL
``` sql
SELECT Title, Release FROM movies
```
  - LINQ
``` csharp
var projected_movies = from m in db.Movies select new {m.Title, m.Release};
```


- In case you do not specify a set of attributes you will get the values of all attributes of that entity. In the following all the attributes values will be selected


   - SQL
``` sql
SELECT * 
FROM movies
```
   - LINQ
   
``` csharp
var projected_movies = from m in db.Movies select m;
```

- You can iterate the result of a query by means of a foreach loop


``` csharp
Console.WriteLine("Movie title | Release");
foreach (var movie in projected_movies){
    Console.WriteLine("- {0} | {1} ", movie.Title, movie.Release);
}

```

### Projection and filtering

- When projecting you can further restrict the values in the result by defining one or more condition. You can define multiple conditions by using logical operators such as AND or OR

- In this step we perform a query that only includes movies which are released after the date 01-01-2000

   - SQL      
``` sql
SELECT *
FROM movies AS m 
WHERE m.Release > 01-01-2000;  
```
   - LINQ
   
``` csharp
var projected_movies =  from m in db.Movies 
                        where m.Release > new DateTime (2000, 1, 1) 
                        select m;
```

### Projection with ordering

- You can also order the result based on the values of a specific one or more columns. The ordering can be descending or ascending

- In following query the return values of the result are movies released after 01-01-2000 and ordered descendly by their release date

   - SQL
   
``` sql
SELECT * 
FROM movies AS m 
WHERE release > '01-01-2000'
ORDER BY m.release DESC; 
```
   - LINQ
   
``` csharp
var projected_movies = from m in db.Movies 
                       where m.Release > new DateTime (2000, 1, 1)
                       orderby m.Release descending 
                       select m;
```

### Grouping and aggregation

- For some cases the resulting values need to be ordered into groups. In the following query we group actors by genders

   - SQL
   
``` sql
SELECT * 
FROM actors
GROUP BY gender; 
```

   - LINQ

``` csharp
var projected_movies = from a in db.Actors
                       group a by a.Gender into genderGroup
                       select genderGroup;
```


- To print the grouped actors use the following code

``` csharp
foreach (var movie in projected_movies){
    Console.WriteLine("+ {0} ", movie.Key);
    foreach (var actor in movie){
        Console.WriteLine("-- {0} ", actor.Name);
    }
}
```

- In combination with aggregation functions you can do additional operations such as counting values of resulting groups. In the following we group by gender and then count the number of actors for each gender

   - SQL
   
``` sql
SELECT gender, count(*) 
FROM actors 
GROUP BY gender; 


```
   - LINQ
   
``` csharp
 var result = from actor in db.Actors
              group actor by actor.Gender into GenderGrp
              select Tuple.Create (
                     GenderGrp.Key,
                     GenderGrp.Count()
              );

```

- Since we change the names of the attributes in the result, we also need to adapt the print to see the output. 


``` csharp
Console.WriteLine("Gender | Number of actors");
foreach (var item in result){
           Console.WriteLine("{0} | {1}", item.Item1, item.Item2);
}
```

### Joining

- When using a join you combine attributes of one or more entities and present the combined values as one single result. In the following we join each movie with its own actors.

   - SQL
   
``` sql
SELECT movies.Title, actors.Name 
FROM movies, actors 
WHERE movies.movieId == actors.movieId 
```
   - LINQ
   
``` csharp
var projected_movies =  from movie in db.Movies
                        from actor in db.Actors
                        where movie.Id == actor.MovieId
                        select new {
                                Title = movie.Title,
                                ActorName = actor.Name
                        };
```

### Subquery and aggregation

- A subquery is useful when you need intermediate result before executing the rest of the query. In LINQ you can implement a subquery by using the keyword `let`. In the following query we implement two queries. The first query will access the data of movies depending on the result of the subquery. The subquery itself access the actors table

   - SQL
   
``` sql
SELECT * 
FROM (SELECT count(a.actorid) AS actorsNr, m.title 
      FROM actors AS a,movies As m 
      WHERE a.movieid = m.movieid 
	  GROUP BY m.title) AS q 
WHERE q.actorsNr < 3;

```
   - LINQ
   
``` csharp
var projected_movies =  from movie in db.Movies
                        let actors_of_movie = (
                            from actor in db.Actors
                            where actor.MovieId == movie.Id
                            select actor)
                        where actors_of_movie.Count() < 3    
                        select new {
                                Title = movie.Title,
                                ActorsCount = actors_of_movie.Count()
                                };
```

- Note that we have a subquery to extract the actors of each movie and we do not use directly the movie.Actors property, because this kind of access is not supported in all versions of .NET core. It is always better to pass through the dbContext.

### Lazy loading

- In the context of EF, lazy loading is the process whereby an entity or collection of entities is automatically loaded from the database the first time that an attribute referring to the entity/entities is accessed. For example, when using the movie entity class defined above, the related actors will be loaded the first time the actors navigation property is accessed. By default LINQ queries use lazy loading to avoid unnecessary resource usage. 

- In LINQ you have a method (toList) that forces immediate query evaluation and returns a `List<T>` that contains the query's result. You can append this method to your query in order to obtain a cached copy of the query results. You can do additional processing of the the data through applying list operations.

- Explicit loading can be used when you are interested in loading the data in memory

- In short try to query at most once the database per problem without nested toList() calls


### Logging (extra material)

- To understand the process of translating LINQ queries into SQL and to understand if the atomic behaviour of the queries is maintained, the EF provides a decorator that logs the queries to a file

``` csharp
using System;
using System.Collections.Generic;
using Microsoft.EntityFrameworkCore;
using Microsoft.Extensions.Logging;
using System.IO;

public class MyLoggerProvider : ILoggerProvider
{
    public ILogger CreateLogger(string categoryName)
    {
        return new MyLogger();
    }

    public void Dispose() { }

    private class MyLogger : ILogger
    {
        public bool IsEnabled(LogLevel logLevel)
        {
            return true;
        }

        public void Log<TState>(LogLevel logLevel, EventId eventId, TState state, Exception exception, Func<TState, Exception, string> formatter)
        {

            File.AppendAllText(@"C:\temp\log.txt", formatter(state, exception)); // change the path to the location you prefer
            Console.WriteLine(formatter(state, exception)); // comment this line if you do not want to also print in the console
        }

        public IDisposable BeginScope<TState>(TState state)
        {
            return null;
        }
    } 
}
```

- To use this decorator in your project you need to connect it to the chosen database connector (postgres, mysql, sqlite, ...) in the file `Model.cs`

``` csharp
...
protected override void OnConfiguring (DbContextOptionsBuilder optionsBuilder) {
            optionsBuilder.UseNpgsql(
            "UserID=postgres;Password=;Host=localhost;Port=5432;Database=MovieDB;Pooling=true;");
        
            //activate logging for compiled queries 
            var lf = new LoggerFactory();
            lf.AddProvider(new MyLoggerProvider());
            optionsBuilder.UseLoggerFactory(lf);
}
```

- Additional resources can be found here: 
    - Source1: https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/keywords/query-keywords
    - Source2: https://docs.microsoft.com/en-us/dotnet/csharp/linq/linq-in-csharpx

